<a href="https://colab.research.google.com/github/bahram3000/test_balancer_app/blob/main/start_deposit_sawp_remove_testnet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!pip install -qqq web3

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 20.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 587.5/587.5 kB 30.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.5/102.5 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.4/51.4 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 340.3/340.3 kB 17.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.8/175.8 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 67.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.4/48.4 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 70.8 MB/s eta 0:00:00


In [ ]:
#weth_usdc_pool_addres='0x6418eec70f50913ff0d756b48d32ce7c02b47c47'
eth_usdc_pool_addres='0x3289680dD4d6C10bb19b899729cda5eEF58AEfF1'

In [5]:
from web3 import Web3
from eth_account import Account

In [6]:
import requests
import json
#import yfinance as yf
import time

In [27]:
def find_key(data, target_key):
    results = []
    if isinstance(data, dict):
        for key, value in data.items():
            if key == target_key:
                results.append(value)
            else:
                results.extend(find_key(value, target_key))
    elif isinstance(data, list):
        for item in data:
            results.extend(find_key(item, target_key))
    return results

In [7]:
RPC = "https://eth-sepolia.g.alchemy.com/v2/demo"
w3 = Web3(Web3.HTTPProvider(RPC))

# کلید خصوصی والت تست‌نت
PRIVATE_KEY = "75074ee5760d9640d993d09bc6e969b37cadcc2c83195886d43b3e86b2b8b6da"
account = Account.from_key(PRIVATE_KEY)
address = account.address

print("Connected:", w3.is_connected())
print("Your address:", address)


Connected: True
Your address: 0x7868EDEfC36A74eba220C75aCFB05c59a4e55822


In [7]:
def check_token_balance(token_list_address:list, inp_address):
  ERC20_ABI = '''
  [
  {"constant":true,"inputs":[{"name":"_owner","type":"address"}],
   "name":"balanceOf","outputs":[{"name":"balance","type":"uint256"}],
   "type":"function"},
  {"constant":true,"inputs":[],"name":"decimals",
   "outputs":[{"name":"","type":"uint8"}],"type":"function"},
  {"constant":true,"inputs":[],"name":"symbol",
   "outputs":[{"name":"","type":"string"}],"type":"function"}
  ]
  '''
  balances = []
  for token in token_list_address:
      contract = w3.eth.contract(address=token, abi=ERC20_ABI)
      bal = contract.functions.balanceOf(inp_address).call()
      dec = contract.functions.decimals().call()
      sym = contract.functions.symbol().call()
      balances.append([sym, bal / (10 ** dec)])
  eth_balance = w3.eth.get_balance(address)
  balances.append(["ETH", w3.from_wei(eth_balance, 'ether')])
  return balances

In [8]:
TOKEN_LIST = [
    "0xfFf9976782d46CC05630D1f6eBAb18b2324d6B14",
    "0x1c7D4B196Cb0C7B01d743Fbc6116a902379C7238",
    "0x1f9840a85d5aF5bf1D1762F925BDADdC4201F984"
]

In [17]:
weth_address="0xfFf9976782d46CC05630D1f6eBAb18b2324d6B14"
usdc_address="0x1c7D4B196Cb0C7B01d743Fbc6116a902379C7238"

In [9]:
check_token_balance(TOKEN_LIST,address)

[['WETH', 0.016],
 ['USDC', 131.561235],
 ['UNI', 0.000830846866961009],
 ['ETH', Decimal('0.080329241200470987')]]

In [10]:
etherscan_apikey = "N68HP1FGB4UQQFBHZQUJIM12HVYWJME96M"
ADDRESS = "0x7868EDEfC36A74eba220C75aCFB05c59a4e55822"
CHAIN_ID = 11155111  # Sepolia

In [11]:
# ۱) موجودی ETH
eth_url = (
    f"https://api.etherscan.io/v2/api?"
    f"chainid={CHAIN_ID}&module=account&action=balance"
    f"&address={ADDRESS}&tag=latest&apikey={etherscan_apikey}"
)

eth_res = requests.get(eth_url).json()
print("ETH balance:", int(eth_res.get("result", 0))/10**18)

# ۲) درخواست لیست توکن — **دقت**: تا جای من اطلاع، Etherscan V2 فعلاً endpoint عمومی برای list token ندارد
tokens_url = (
    f"https://api.etherscan.io/v2/api?"
    f"chainid={CHAIN_ID}&module=account&action=tokenlist"
    f"&address={ADDRESS}&apikey={etherscan_apikey}"
)

tokens_res = requests.get(tokens_url).json()
print("Tokens response:", tokens_res)


ETH balance: 0.08032924120047098
Tokens response: {'status': '0', 'message': 'NOTOK', 'result': 'Error! Missing Or invalid Action name'}


In [12]:
# آدرس رسمی WETH روی Sepolia
WETH = Web3.to_checksum_address("0xfFf9976782d46CC05630D1f6eBAb18b2324d6B14")

# ABI اصلی توابع deposit/withdraw
WETH_ABI = json.loads("""
[
    {
        "constant": false,
        "inputs": [],
        "name": "deposit",
        "outputs": [],
        "payable": true,
        "stateMutability": "payable",
        "type": "function"
    },
    {
        "constant": false,
        "inputs": [{"name":"wad","type":"uint256"}],
        "name": "withdraw",
        "outputs": [],
        "payable": false,
        "stateMutability": "nonpayable",
        "type": "function"
    }
]
""")

contract = w3.eth.contract(address=WETH, abi=WETH_ABI)

# ------------------------------
# مقدار ETH که می‌خواهی Wrap کنی
# ------------------------------
amount_eth = 0.001
amount_wei = w3.to_wei(amount_eth, "ether")

# ------------------------------
# ساخت تراکنش wrap → deposit()
# ------------------------------

tx = contract.functions.deposit().build_transaction({
    "from": address,
    "value": amount_wei,
    "nonce": w3.eth.get_transaction_count(address),
    "chainId": 11155111,
})

estimated_gas = w3.eth.estimate_gas(tx)
print("Estimated gas:", estimated_gas)




estimated_gas = w3.eth.estimate_gas(tx)
print("Estimated gas:", estimated_gas)

# gas_used که گرفتیم
gas_used = estimated_gas

eth_price = requests.get("https://api.coingecko.com/api/v3/simple/price?ids=ethereum&vs_currencies=usd").json()["ethereum"]["usd"]

# gas_price شبکه (Wei)
gas_price = w3.eth.gas_price  # مثال: 5 gwei = 5_000_000_000

# سه حالت
gas_price_slow   = int(gas_price * 0.7)
gas_price_normal = gas_price
gas_price_fast   = int(gas_price * 1.3)

cost_slow   = gas_used * gas_price_slow * eth_price / 1e18
cost_normal = gas_used * gas_price_normal * eth_price / 1e18
cost_fast   = gas_used * gas_price_fast * eth_price / 1e18

print("Slow cost  USD:", cost_slow)
print("Normal cost USD:", cost_normal)
print("Fast cost  USD:", cost_fast)

Estimated gas: 28284
Estimated gas: 28284
Slow cost  USD: 6.199295001958848e-05
Normal cost USD: 8.856139512538272e-05
Fast cost  USD: 0.00011512975167057888


In [ ]:
tx = contract.functions.deposit().build_transaction({
    "from": address,
    "value": amount_wei,
    "gas": 150000,               # safe limit
    "gasPrice": gas_price_slow,
    "nonce": w3.eth.get_transaction_count(address),
    "chainId": 11155111          # chainId سپولیا
})


# فرض RPC و tx ساخته شده است
signed_txn = w3.eth.account.sign_transaction(tx, private_key=PRIVATE_KEY)

# ارسال تراکنش
tx_hash = w3.eth.send_raw_transaction(signed_txn.raw_transaction)

print("TX sent:", tx_hash.hex())
print("Track here: https://sepolia.etherscan.io/tx/" +'0x' +tx_hash.hex())

TX sent: 17adce6013d93e0d124785977f0ecbec735f0c9563a7539ebbc7828bfc44b45d
Track here: https://sepolia.etherscan.io/tx/0x17adce6013d93e0d124785977f0ecbec735f0c9563a7539ebbc7828bfc44b45d


In [13]:
pool = w3.eth.contract(
    address=eth_usdc_pool_addres,
    abi=[
        {"constant":True,"inputs":[],"name":"token0","outputs":[{"name":"","type":"address"}],"type":"function"},
        {"constant":True,"inputs":[],"name":"token1","outputs":[{"name":"","type":"address"}],"type":"function"}
    ]
)

try:
    t0 = pool.functions.token0().call()
    t1 = pool.functions.token1().call()
    print("Pool tokens:", t0, t1)
except Exception as e:
    print("Not a valid pool or cannot read:", e)


Pool tokens: 0x1c7D4B196Cb0C7B01d743Fbc6116a902379C7238 0xfFf9976782d46CC05630D1f6eBAb18b2324d6B14


In [ ]:
# -------- آدرس‌ها
WETH = Web3.to_checksum_address("0xfFf9976782d46CC05630D1f6eBAb18b2324d6B14")
USDC = Web3.to_checksum_address("0x1c7D4B196Cb0C7B01d743Fbc6116a902379C7238")
ROUTER = Web3.to_checksum_address("0xE592427A0AEce92De3Edee1F18E0157C05861564")  # Uniswap v3 Router

# -------- ABI Router v3: exactInputSingle
router_abi = json.loads("""
[
  {
    "inputs": [
      {
        "components": [
          {"internalType": "address","name": "tokenIn","type": "address"},
          {"internalType": "address","name": "tokenOut","type": "address"},
          {"internalType": "uint24","name": "fee","type": "uint24"},
          {"internalType": "address","name": "recipient","type": "address"},
          {"internalType": "uint256","name": "deadline","type": "uint256"},
          {"internalType": "uint256","name": "amountIn","type": "uint256"},
          {"internalType": "uint256","name": "amountOutMinimum","type": "uint256"},
          {"internalType": "uint160","name": "sqrtPriceLimitX96","type": "uint160"}
        ],
        "internalType": "struct ISwapRouter.ExactInputSingleParams",
        "name": "params",
        "type": "tuple"
      }
    ],
    "name": "exactInputSingle",
    "outputs": [{"internalType": "uint256","name": "amountOut","type": "uint256"}],
    "stateMutability": "payable",
    "type": "function"
  }
]
""")

router = w3.eth.contract(address=ROUTER, abi=router_abi)

# -------- پارامترهای تراکنش
amount_in_eth = 0.002
amount_in_wei = w3.to_wei(amount_in_eth, "ether")

fee = 500
recipient = address
deadline = int(time.time()) + 600  # 10 دقیقه

amount_out_min = 1  # حداقل USDC دریافتی (slippage باید محاسبه شود)
sqrtPriceLimitX96 = 0  # بدون محدودیت قیمت

params = (
    WETH,
    USDC,
    fee,
    recipient,
    deadline,
    amount_in_wei,
    amount_out_min,
    sqrtPriceLimitX96
)

# -------- Gas
gas_price = int(w3.eth.gas_price * 0.7)  # slow
gas_limit = 250000

# -------- ساخت تراکنش
tx = router.functions.exactInputSingle(params).build_transaction({
    "from": address,
    "value": amount_in_wei,
    "gas": gas_limit,
    "gasPrice": gas_price,
    "nonce": w3.eth.get_transaction_count(address),
    "chainId": 11155111
})

# -------- امضا و ارسال
signed_tx = w3.eth.account.sign_transaction(tx, PRIVATE_KEY)
tx_hash = w3.eth.send_raw_transaction(signed_tx.raw_transaction)

print("TX sent:", tx_hash.hex())
print("Track here: https://sepolia.etherscan.io/tx/" +'0x'+ tx_hash.hex())


TX sent: 3d7afa45540363d156ad085980c72b098060d15219c5fb2437a4da046c4e086e
Track here: https://sepolia.etherscan.io/tx/0x3d7afa45540363d156ad085980c72b098060d15219c5fb2437a4da046c4e086e


In [ ]:
check_token_balance(TOKEN_LIST,address)

[['WETH', 0.002], ['USDC', 20.242914]]

In [ ]:
receipt = w3.eth.wait_for_transaction_receipt(tx_hash)
print("Transaction status:", receipt.status)

Transaction status: 1


In [ ]:
eth_balance = w3.eth.get_balance(address)
print("ETH Balance:", w3.from_wei(eth_balance, 'ether'))

ETH Balance: 0.081737706151980272


In [ ]:
check_token_balance(TOKEN_LIST,address)

[['WETH', 0.002], ['USDC', 20.242914]]

In [ ]:
#sepolia_subgraph_id="DmMXLtMZnGbQXASJ7p1jfzLUbBYnYUD9zNBTxpkjHYXV"
#sepolia_subgraph_id="FddwGma9QoedFcxYwrMU8qq9Lsd5PS2XqjWSLNRMPKLn"
#GRAPH_API_KEY='0f011cf8766c88189291aa0b95792828'

In [ ]:
#subgraph_url= f"https://gateway.thegraph.com/api/{GRAPH_API_KEY}/subgraphs/id/{sepolia_subgraph_id}"

In [14]:
# --- ABI لازم فقط برای توابع مهم ---
POOL_ABI = [
    {"inputs":[],"name":"slot0","outputs":[
        {"internalType":"uint160","name":"sqrtPriceX96","type":"uint160"},
        {"internalType":"int24","name":"tick","type":"int24"},
        {"internalType":"uint16","name":"observationIndex","type":"uint16"},
        {"internalType":"uint16","name":"observationCardinality","type":"uint16"},
        {"internalType":"uint16","name":"observationCardinalityNext","type":"uint16"},
        {"internalType":"uint8","name":"feeProtocol","type":"uint8"},
        {"internalType":"bool","name":"unlocked","type":"bool"}
    ],"stateMutability":"view","type":"function"},

    {"inputs":[],"name":"liquidity","outputs":[
        {"internalType":"uint128","name":"","type":"uint128"}
    ],"stateMutability":"view","type":"function"},

    {"inputs":[],"name":"token0","outputs":[
        {"internalType":"address","name":"","type":"address"}
    ],"stateMutability":"view","type":"function"},

    {"inputs":[],"name":"token1","outputs":[
        {"internalType":"address","name":"","type":"address"}
    ],"stateMutability":"view","type":"function"},

    {"inputs":[],"name":"fee","outputs":[
        {"internalType":"uint24","name":"","type":"uint24"}
    ],"stateMutability":"view","type":"function"},
]

# --- آدرس استخر (Pool Address) را اینجا بگذار ---

pool = w3.eth.contract(address=eth_usdc_pool_addres, abi=POOL_ABI)

# --- گرفتن اطلاعات ---
slot0 = pool.functions.slot0().call()
liquidity = pool.functions.liquidity().call()
token0 = pool.functions.token0().call()
token1 = pool.functions.token1().call()
fee = pool.functions.fee().call()

print("slot0 =", slot0)
print("liquidity =", liquidity)
print("token0 =", token0)
print("token1 =", token1)
print("fee =", fee)


slot0 = [858150196932505559061578568269596, 185813, 15, 56, 56, 0, True]
liquidity = 11475381455869365
token0 = 0x1c7D4B196Cb0C7B01d743Fbc6116a902379C7238
token1 = 0xfFf9976782d46CC05630D1f6eBAb18b2324d6B14
fee = 500


In [40]:
WALLET_ADDRESS = "0x7868EDEfC36A74eba220C75aCFB05c59a4e55822"

# 2. Network Config
CHAIN_ID = 11155111 # Sepolia

# 3. Router Address (از اسکرین‌شات شما)
SWAP_ROUTER_ADDRESS = "0x3bFA4769FB09eefC5a80d6E87c3B9C650f7Ae48E"

# 4. Token Addresses (Sepolia)
WETH_ADDRESS = "0xfFf9976782d46CC05630D1f6eBAb18b2324d6B14"
USDC_ADDRESS = "0x1c7D4B196Cb0C7B01d743Fbc6116a902379C7238"
POOL_ADDRESS = "0x3289680dD4d6C10bb19b899729cda5eEF58AEfF1" # Pool for reading fee

# 5. Swap Amount
AMOUNT_IN_ETH = 0.002

# ---------------- ABIs ----------------
# 1. ERC20 / WETH ABI
ERC20_ABI = json.loads('''[
    {"constant":true,"inputs":[{"name":"_owner","type":"address"}],"name":"balanceOf","outputs":[{"name":"balance","type":"uint256"}],"type":"function"},
    {"constant":true,"inputs":[],"name":"decimals","outputs":[{"name":"","type":"uint8"}],"type":"function"},
    {"constant":false,"inputs":[{"name":"_spender","type":"address"},{"name":"_value","type":"uint256"}],"name":"approve","outputs":[{"name":"","type":"bool"}],"type":"function"},
    {"constant":true,"inputs":[{"name":"_owner","type":"address"},{"name":"_spender","type":"address"}],"name":"allowance","outputs":[{"name":"","type":"uint256"}],"type":"function"},
    {"inputs":[],"name":"deposit","outputs":[],"stateMutability":"payable","type":"function"}
]''')

# 2. POOL ABI (Just to read fee)
POOL_ABI = json.loads('''[
    {"inputs":[],"name":"fee","outputs":[{"internalType":"uint24","name":"","type":"uint24"}],"stateMutability":"view","type":"function"}
]''')

# 3. SWAP ROUTER ABI (MATCHING YOUR SCREENSHOT - 7 PARAMS, NO DEADLINE)
ROUTER_ABI = json.loads('''[
    {
        "inputs": [
            {
                "components": [
                    {"internalType": "address", "name": "tokenIn", "type": "address"},
                    {"internalType": "address", "name": "tokenOut", "type": "address"},
                    {"internalType": "uint24", "name": "fee", "type": "uint24"},
                    {"internalType": "address", "name": "recipient", "type": "address"},
                    {"internalType": "uint256", "name": "amountIn", "type": "uint256"},
                    {"internalType": "uint256", "name": "amountOutMinimum", "type": "uint256"},
                    {"internalType": "uint160", "name": "sqrtPriceLimitX96", "type": "uint160"}
                ],
                "internalType": "struct IV3SwapRouter.ExactInputSingleParams",
                "name": "params",
                "type": "tuple"
            }
        ],
        "name": "exactInputSingle",
        "outputs": [{"internalType": "uint256", "name": "amountOut", "type": "uint256"}],
        "stateMutability": "payable",
        "type": "function"
    }
]''')

# ---------------- SETUP ----------------
w3 = Web3(Web3.HTTPProvider(RPC))
wallet = w3.to_checksum_address(WALLET_ADDRESS)
router_addr = w3.to_checksum_address(SWAP_ROUTER_ADDRESS)
weth_addr = w3.to_checksum_address(WETH_ADDRESS)
usdc_addr = w3.to_checksum_address(USDC_ADDRESS)
pool_addr = w3.to_checksum_address(POOL_ADDRESS)

weth = w3.eth.contract(address=weth_addr, abi=ERC20_ABI)
usdc = w3.eth.contract(address=usdc_addr, abi=ERC20_ABI)
router = w3.eth.contract(address=router_addr, abi=ROUTER_ABI)
pool = w3.eth.contract(address=pool_addr, abi=POOL_ABI)

# ---------------- HELPERS ----------------
def human_to_raw(amount, decimals):
    return int(decimal.Decimal(str(amount)) * (decimal.Decimal(10) ** decimals))

def raw_to_human(raw, decimals):
    return decimal.Decimal(raw) / (decimal.Decimal(10) ** decimals)

def build_send_wait(function_call, value=0):
    try:
        nonce = w3.eth.get_transaction_count(wallet)
        tx_params = {
            'from': wallet,
            'nonce': nonce,
            'value': value,
            'chainId': CHAIN_ID,
            'gasPrice': w3.eth.gas_price
        }

        # Build transaction
        tx = function_call.build_transaction(tx_params)

        # Estimate gas (safety margin)
        try:
            gas_est = w3.eth.estimate_gas(tx)
            tx['gas'] = int(gas_est * 1.2)
        except:
            tx['gas'] = 500000 # Fallback

        # Sign and Send
        signed_tx = w3.eth.account.sign_transaction(tx, PRIVATE_KEY)
        tx_hash = w3.eth.send_raw_transaction(signed_tx.raw_transaction)
        print(f"   >>> TX Sent: {tx_hash.hex()}")

        # Wait for receipt
        receipt = w3.eth.wait_for_transaction_receipt(tx_hash, timeout=120)
        return receipt
    except Exception as e:
        print(f"   >>> ERROR in Transaction: {e}")
        return None

# ---------------- MAIN ----------------
def main():
    if not w3.is_connected():
        print("RPC Not Connected!")
        return

    print(f"\n--- STARTING SWAP ---")
    print(f"Router: {SWAP_ROUTER_ADDRESS}")
    print(f"Wallet: {WALLET_ADDRESS}")

    # 1. Get Fee
    try:
        fee = pool.functions.fee().call()
        print(f"Pool Fee: {fee}")
    except:
        fee = 500 # Default fallback
        print("Could not read fee, using default 500")

    # 2. Deposit ETH -> WETH
    amount_in_wei = human_to_raw(AMOUNT_IN_ETH, 18)
    weth_bal = weth.functions.balanceOf(wallet).call()

    print(f"\n1) Checking WETH Balance...")
    if weth_bal < amount_in_wei:
        print(f"   Insufficient WETH. Depositing {AMOUNT_IN_ETH} ETH...")
        receipt = build_send_wait(weth.functions.deposit(), value=amount_in_wei)
        if not receipt or receipt.status != 1:
            print("   Deposit Failed!")
            return
    else:
        print("   WETH balance sufficient.")

    # 3. Approve Router
    print(f"\n2) Checking Approval...")
    allowance = weth.functions.allowance(wallet, router_addr).call()
    if allowance < amount_in_wei:
        print("   Approving Router...")
        receipt = build_send_wait(weth.functions.approve(router_addr, 2**256 - 1))
        if not receipt or receipt.status != 1:
            print("   Approval Failed!")
            return
    else:
        print("   Already approved.")

    # 4. EXECUTE SWAP
    print(f"\n3) Executing Swap (exactInputSingle)...")

    # --- HERE IS THE FIX BASED ON YOUR SCREENSHOT ---
    # No 'deadline' key is included
    params = {
        "tokenIn": weth_addr,
        "tokenOut": usdc_addr,
        "fee": fee,
        "recipient": wallet,
        "amountIn": amount_in_wei,
        "amountOutMinimum": 0,       # Slippage 100% allowed for testing
        "sqrtPriceLimitX96": 0
    }

    print(f"   Params: {params}")

    receipt = build_send_wait(router.functions.exactInputSingle(params))

    if receipt and receipt.status == 1:
        print("\n✅ SWAP SUCCESSFUL! Transaction Confirmed.")
        print(f"   Hash: {receipt.transactionHash.hex()}")

        print("   Waiting 15 seconds for RPC update to check balances...")
        time.sleep(15) # Wait for RPC to sync

        usdc_bal = usdc.functions.balanceOf(wallet).call()
        print(f"   Final USDC Balance: {raw_to_human(usdc_bal, 6)}")
    else:
        print("\n❌ SWAP FAILED or Reverted.")

if __name__ == "__main__":
    main()


--- STARTING SWAP ---
Router: 0x3bFA4769FB09eefC5a80d6E87c3B9C650f7Ae48E
Wallet: 0x7868EDEfC36A74eba220C75aCFB05c59a4e55822
Pool Fee: 500

1) Checking WETH Balance...
   WETH balance sufficient.

2) Checking Approval...
   Already approved.

3) Executing Swap (exactInputSingle)...
   Params: {'tokenIn': '0xfFf9976782d46CC05630D1f6eBAb18b2324d6B14', 'tokenOut': '0x1c7D4B196Cb0C7B01d743Fbc6116a902379C7238', 'fee': 500, 'recipient': '0x7868EDEfC36A74eba220C75aCFB05c59a4e55822', 'amountIn': 2000000000000000, 'amountOutMinimum': 0, 'sqrtPriceLimitX96': 0}
   >>> TX Sent: 99a0fbcfa9a9114a241275d1d05880158626ebaccd18cb5241f76947b67d16fa

✅ SWAP SUCCESSFUL! Transaction Confirmed.
   Hash: 99a0fbcfa9a9114a241275d1d05880158626ebaccd18cb5241f76947b67d16fa
   Waiting 15 seconds for RPC update to check balances...
   Final USDC Balance: 165.719236


In [41]:
check_token_balance(TOKEN_LIST,address)

[['WETH', 0.044],
 ['USDC', 165.719236],
 ['UNI', 0.000830846866961009],
 ['ETH', Decimal('0.052328485126705575')]]

In [38]:
address

'0x7868EDEfC36A74eba220C75aCFB05c59a4e55822'

In [15]:
if not w3.is_connected():
    raise SystemExit("اتصال برقرار نشد")

# آدرس صحیح NonfungiblePositionManager در شبکه Sepolia
nfpm_address="0x1238536071E1c677A632429e3655c799b22cDA52"
POSITION_MANAGER_ADDRESS = w3.to_checksum_address(nfpm_address)

POSITION_MANAGER_ABI = [
    {
        "inputs":[{"internalType":"address","name":"owner","type":"address"}],
        "name":"balanceOf",
        "outputs":[{"internalType":"uint256","name":"","type":"uint256"}],
        "stateMutability":"view",
        "type":"function"
    },
    {
        "inputs":[{"internalType":"uint256","name":"tokenId","type":"uint256"}],
        "name":"positions",
        "outputs":[
            {"internalType":"uint96","name":"nonce","type":"uint96"},
            {"internalType":"address","name":"operator","type":"address"},
            {"internalType":"address","name":"token0","type":"address"},
            {"internalType":"address","name":"token1","type":"address"},
            {"internalType":"uint24","name":"fee","type":"uint24"},
            {"internalType":"int24","name":"tickLower","type":"int24"},
            {"internalType":"int24","name":"tickUpper","type":"int24"},
            {"internalType":"uint128","name":"liquidity","type":"uint128"},
            {"internalType":"uint256","name":"feeGrowthInside0LastX128","type":"uint256"},
            {"internalType":"uint256","name":"feeGrowthInside1LastX128","type":"uint256"},
            {"internalType":"uint128","name":"tokensOwed0","type":"uint128"},
            {"internalType":"uint128","name":"tokensOwed1","type":"uint128"}
        ],
        "stateMutability":"view",
        "type":"function"
    },
    {
        "inputs":[{"internalType":"address","name":"owner","type":"address"},{"internalType":"uint256","name":"index","type":"uint256"}],
        "name":"tokenOfOwnerByIndex",
        "outputs":[{"internalType":"uint256","name":"","type":"uint256"}],
        "stateMutability":"view",
        "type":"function"
    }
]

pm_contract = w3.eth.contract(address=POSITION_MANAGER_ADDRESS, abi=POSITION_MANAGER_ABI)

print(f"Checking positions for: {address}")

try:
    # 1. گرفتن تعداد NFTها
    balance = pm_contract.functions.balanceOf(address).call()
    print(f"تعداد NFT پوزیشن‌ها: {balance}")

    # 2. گرفتن اطلاعات هر NFT
    if balance > 0:
        for i in range(balance):
            # گرفتن Token ID بر اساس ایندکس
            token_id = pm_contract.functions.tokenOfOwnerByIndex(address, i).call()

            # گرفتن جزئیات پوزیشن
            pos = pm_contract.functions.positions(token_id).call()

            print(f"\n--- Position #{i+1} (Token ID: {token_id}) ---")

            print(f"Token0: {pos[2]}")
            print(f"Token1: {pos[3]}")
            print(f"Fee Tier: {pos[4]}")
            print(f"Liquidity: {pos[7]}")
            print(f"Tick Range: [{pos[5]} - {pos[6]}]")
            print(pos[7:])
    else:
        print("هیچ پوزیشنی یافت نشد.")

except Exception as e:
    print(f"خطا در خواندن اطلاعات: {e}")

Checking positions for: 0x7868EDEfC36A74eba220C75aCFB05c59a4e55822
تعداد NFT پوزیشن‌ها: 1

--- Position #1 (Token ID: 217011) ---
Token0: 0x1c7D4B196Cb0C7B01d743Fbc6116a902379C7238
Token1: 0x1f9840a85d5aF5bf1D1762F925BDADdC4201F984
Fee Tier: 500
Liquidity: 0
Tick Range: [142500 - 156410]
[0, 3699105600461476284041382282722, 46272342038586624227567994123812579157, 0, 0]


In [18]:
pm_contract.functions.tokenOfOwnerByIndex(address, 0).call()

217011

In [19]:
v3factoryaddress="0x0227628f3F023bb0B980b67D528571c95c6DaC1c"
FACTORY_ADDRESS = w3.to_checksum_address(v3factoryaddress)
FACTORY_ABI = [
    {
        "inputs": [
            {"internalType":"address","name":"tokenA","type":"address"},
            {"internalType":"address","name":"tokenB","type":"address"},
            {"internalType":"uint24","name":"fee","type":"uint24"}
        ],
        "name":"getPool",
        "outputs":[{"internalType":"address","name":"pool","type":"address"}],
        "stateMutability":"view",
        "type":"function"
    }
]
for i in range(balance):

  token_id = pm_contract.functions.tokenOfOwnerByIndex(address, i).call()
  pos = pm_contract.functions.positions(token_id).call()
  token0 = pos[2]
  token1 = pos[3]
  fee = pos[4]
  factory = w3.eth.contract(address=FACTORY_ADDRESS, abi=FACTORY_ABI)
  pool_address = factory.functions.getPool(token0, token1, fee).call()
  print(f"Pool Address: {pool_address}")


Pool Address: 0x6E81F4a9bE879CE9FCBAD4Eafc9311fB9e766299
